# Simulation and verification

When modelling an algorithm for output to a target language or hardware synthesis, one would - in Python - typically begin with a classic set of functions that are executed sequentially.

For emission to hardware, the corresponding logic and signal structure is conceived, a timing/clock concept and state machines or pipelines introduced.

Then, the synthesizeable hardware should do the same as the prototyped algorithm.

This is where verification comes into play. The IRL language provides an `.evaluate()` method for generators and expressions, so that functionality can be verified - inside the Python environment.

Then, verification of correct functionality may occur at later stages:
  1. Post HDL translation: correct functional description for a synthesis tool
  2. Post mapping: Correct inference to logic elements, verification based on vendor element primitives
  3. Post placement: Simulation including gate timings
  
This chapter will only elaborate on the means to do your own verification. For instance, you may want to run a Python model and a synthesized edition of it in parallel.

## A simple simulator dummy

A simulation normally creates external stimuli of a virtual hardware model via a sequence. It can also be in fact a working hardware counterpart and the testbench would then only consist of clocks, reset and a data I/O FIFO to communicate with external software. However, a unit test typically requires a seuquential stimulus approach.

To elaborate, we create a simple sequence using a simulator dummy to see what happens:

In [1]:
from myirl.targets.dummy import DummySimulator
from cyhdl import *

sim = DummySimulator()

a = sim.Signal(intbv()[5:].signed())
b = sim.Signal(intbv()[8:])

We have to use a `sim.Signal` for this, because we are handling a simulation from native python, and not generating logic at this point.

In [2]:
a.next  = 4
b.next = 0

If we evaluate the signal value at this point, we will get an error:

In [3]:
try:
    a.evaluate()
except Exception as e:
    print(e)

Signal `s_c209` (or element) not initialized. Refusing to Co-Simulate.


The reason is, that the signal is a priori uninitialized and will get its value upon the next simulation time step:

In [4]:
sim.proceed(1)

In [5]:
int(a), int(b)

(4, 0)

This introduces an unusual aspect for a sequential minded programmer. Values would, for instance, swap using the following code:

In [6]:
a.next = b
b.next = a
sim.proceed(1)
int(a), int(b)

(0, 4)

We however note, that this is only valid in hardware when inside a synchronous process, that executes this action only at a particular clock edge. Otherwise we would create combinatorial loops.

## A test bench sequence

To test a few basic operations in a sequential way, we can create this simple test bench:

In [7]:
def testbench(a, b):
    a.next = 3
    b.next = 5
    yield 1
    a.next = b
    b.next = a
    assert a == 3 and b == 5
    yield 1 # Values are now swapped
    assert a == 5 and b == 3
    
    a.next  = a * b
    print("Done multiplicating")
    yield 1

We pass this to the simulator dummy:

In [8]:
u, v = [ sim.Signal(intbv()[8:], name = n) for n in "uv" ]
sim.handle_sequence(testbench(u, v))

print(u, v)

[ STEP: 1 -> 3 ]
[ STEP: 1 -> 4 ]
Done multiplicating
[ STEP: 1 -> 5 ]
<u> : 0x0f <v> : 0x03


Note that repeated calling of the simulator handler will increase the current time and also maintain the state of the signals. It is up to the simulator, to actually update a signal change upon the next time step.

## Co-Simulation versus simulation

This simulation clearly runs in native Python `as is`, using explicit time steps given by yield. This is the only main stimulus here (and we can only have one, for this dummy simulator).

However for a real parallel-alike simulation, we these stimuli need to propagate into hardware. There are several methods to achieve this:

* Translate the entire hardware and simulation sequence to a HDL and execute
* Translate only the hardware to a simulation kernel target and drive it externally from a Python native sequence

The latter is referred to as co-simulation. From the top level, we have Python signals, inside synthesizeable hardware, we map them to their corresponding entities.

The cyrite simulation concept follows the goal to allow switching between simulation domains without having to rewrite the code.
However, this also implies that you don't always see what you get, because code is interpreted in two ways, like a testbench `@sequence` construct will turn out:
* in a silently translated generator to emit HDL
* to be a natively run co-routine interacting with a CoSimulator back end

In any case, a simulation is not always portable, and you might be bound to one specific simulator.

A few examples scenarios:


* VHDL only, timing accurate simulation required: ghdl.GHDL simulator interface
* Verilog only, timing accurate: icarus.ICARUS
* All HDL, but Co-Simulation: yosys.CXXRTL
   * Hardware elements emitted as executable hardware emulation
   * Timing specific elements in Python native domain
   
Further reading: [Simulation](simulation.ipynb)